In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

In [2]:
import sys
print(sys.path) 
sys.path.append("/cellar/users/mpagadal/Programs/anaconda3/lib/python3.7/site-packages")
sys.path.insert(1, '/cellar/users/mpagadal/Data/gcta/scripts')
import json

['/data/cellardata/users/mpagadal/projects/germline-immune/heritability/notebooks', '/nrnb/opt/anaconda3.7/lib/python37.zip', '/nrnb/opt/anaconda3.7/lib/python3.7', '/nrnb/opt/anaconda3.7/lib/python3.7/lib-dynload', '', '/cellar/users/mpagadal/.local/lib/python3.7/site-packages', '/nrnb/opt/anaconda3.7/lib/python3.7/site-packages', '/nrnb/opt/anaconda3.7/lib/python3.7/site-packages/IPython/extensions', '/cellar/users/mpagadal/.ipython']


In [3]:
from compile_gcta_df import *

## Get nonhla and nonkir data

In [4]:
def fdr(df,pseudo):
    #make heritability estimate numeric
    df["V(G)/Vp"]=pd.to_numeric(df["V(G)/Vp"])
    #add pseudocount
    if pseudo == True:
        df["Pval"]=df["Pval"]+0.0000001
    
    #calculate benjamin-hochberg fdr
    genes=df["gene"].tolist()
    probabilities = df["Pval"].to_numpy()
    report = multi.multipletests(probabilities,
                alpha=0.05,
                method="fdr_bh",
                is_sorted=False,
            )
    discoveries = report[1]
    df["fdr"]=discoveries
    df["fdr"]=pd.to_numeric(df["fdr"])
    df["-log10(fdr)"]=-np.log10(df["fdr"])
    
    return(df)

In [5]:
results=[""]

In [7]:
nonhla_tpm=compile_gcta_results(results,output="overall",direct="/cellar/users/mpagadal/Data/projects/germline-immune/heritability/results/tpm/no-hla-kir")
nonhla_firebrowse=compile_gcta_results(results,output="overall",direct="/cellar/users/mpagadal/Data/projects/germline-immune/heritability/results/firebrowse/no-hla-kir")
nonhla_pancan=compile_gcta_results(results,output="overall",direct="/cellar/users/mpagadal/Data/projects/germline-immune/heritability/results/pancanatlas/no-hla-kir")

compiling gene level heritability dataframe.....


/cellar/users/mpagadal/Data/gcta/scripts/compile_gcta_df.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variance["V(G) SE"]=se[0].tolist()
/cellar/users/mpagadal/Data/gcta/scripts/compile_gcta_df.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variance["V(e) SE"]=se[1].tolist()
/cellar/users/mpagadal/Data/gcta/scripts/compile_gcta_df.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

compiling gene level heritability dataframe.....
compiling gene level heritability dataframe.....


In [8]:
hla_tpm=compile_gcta_results(results,output="overall",direct="/cellar/users/mpagadal/Data/projects/germline-immune/heritability/results/tpm/hla-kir")
hla_firebrowse=compile_gcta_results(results,output="overall",direct="/cellar/users/mpagadal/Data/projects/germline-immune/heritability/results/firebrowse/hla-kir")
hla_pancan=compile_gcta_results(results,output="overall",direct="/cellar/users/mpagadal/Data/projects/germline-immune/heritability/results/pancanatlas/hla-kir")

compiling gene level heritability dataframe.....
compiling gene level heritability dataframe.....
compiling gene level heritability dataframe.....


In [10]:
print(nonhla_tpm.shape)
print(nonhla_firebrowse.shape)
print(nonhla_pancan.shape)

(746, 16)
(738, 16)
(741, 16)


In [11]:
pseudocount=True
nonhla_tpm=fdr(nonhla_tpm,pseudocount)
nonhla_firebrowse=fdr(nonhla_firebrowse,pseudocount)
nonhla_pancan=fdr(nonhla_pancan,pseudocount)

In [12]:
pseudocount=True
hla_tpm=fdr(hla_tpm,pseudocount)
hla_firebrowse=fdr(hla_firebrowse,pseudocount)
hla_pancan=fdr(hla_pancan,pseudocount)

In [13]:
nonhla_tpm=nonhla_tpm.rename(columns={"V(G)/Vp":"tpm V(G)/V(p)","-log10(fdr)":"tpm -log10(fdr)"})
nonhla_firebrowse=nonhla_firebrowse.rename(columns={"V(G)/Vp":"firebrowse V(G)/V(p)","-log10(fdr)":"firebrowse -log10(fdr)"})
nonhla_pancan=nonhla_pancan.rename(columns={"V(G)/Vp":"pancan V(G)/V(p)","-log10(fdr)":"pancan -log10(fdr)"})


In [14]:
hla_tpm=hla_tpm.rename(columns={"V(G)/Vp":"tpm V(G)/V(p)","-log10(fdr)":"tpm -log10(fdr)"})
hla_firebrowse=hla_firebrowse.rename(columns={"V(G)/Vp":"firebrowse V(G)/V(p)","-log10(fdr)":"firebrowse -log10(fdr)"})
hla_pancan=hla_pancan.rename(columns={"V(G)/Vp":"pancan V(G)/V(p)","-log10(fdr)":"pancan -log10(fdr)"})


In [15]:
total_nonhla=pd.merge(nonhla_tpm[["tpm V(G)/V(p)","gene","tpm -log10(fdr)"]],nonhla_firebrowse[["firebrowse V(G)/V(p)","gene","firebrowse -log10(fdr)"]], on="gene",how="outer")
total_nonhla=pd.merge(total_nonhla,nonhla_pancan[["pancan V(G)/V(p)","gene","pancan -log10(fdr)"]],on="gene",how="outer")


In [16]:
total_hla=pd.merge(hla_tpm[["tpm V(G)/V(p)","gene","tpm -log10(fdr)"]],hla_firebrowse[["firebrowse V(G)/V(p)","gene","firebrowse -log10(fdr)"]], on="gene",how="outer")
total_hla=pd.merge(total_hla,hla_pancan[["pancan V(G)/V(p)","gene","pancan -log10(fdr)"]],on="gene",how="outer")

In [17]:
total_nonhla["consensus V(g)/V(p)"]=total_nonhla[["firebrowse V(G)/V(p)","pancan V(G)/V(p)"]].apply(lambda x: len([x for x in list(x) if x>0.05]), axis=1)
total_nonhla["consensus fdr"]=total_nonhla[["firebrowse -log10(fdr)","pancan -log10(fdr)"]].apply(lambda x: len([x for x in list(x) if x>1]), axis=1)

In [18]:
total_hla["consensus V(g)/V(p)"]=total_hla[["firebrowse V(G)/V(p)","pancan V(G)/V(p)"]].apply(lambda x: len([x for x in list(x) if x>0.05]), axis=1)
total_hla["consensus fdr"]=total_hla[["firebrowse -log10(fdr)","pancan -log10(fdr)"]].apply(lambda x: len([x for x in list(x) if x>1]), axis=1)

In [21]:
total_nonhla.to_csv("../data/nonhla_gcta_results.tsv",sep="\t",index=None)
total_hla.to_csv("../data/hla_gcta_results.tsv",sep="\t",index=None)